In [1]:
!pip install backtrader
!pip install yfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 5.7 MB/s eta 0:00:00


이동평균선 교차 전략

In [2]:
from datetime import datetime
import backtrader as bt
import math
import yfinance as yf

# SMA 전략
class SmaCross(bt.Strategy):
    params = dict(
        pfast=30,  # 단기 SMA
        pslow=200  # 장기 SMA
    )

    def __init__(self):
        self.sma1 = bt.ind.SMA(period=self.p.pfast) # 단기 SMA
        self.sma2 = bt.ind.SMA(period=self.p.pslow) # 장기 SMA

    # 인덱싱(self.sma1[0], self.sma1[-1])의 의미
    # self.sma1[0]: 가장 최근 시점의 SMA 값을 의미합니다. 인덱스 0은 현재 시점의 데이터를 나타냅니다.
    # self.sma1[-1]: 바로 이전 시점의 SMA 값을 의미합니다. 인덱스 -1은 이전 날 또는 이전 데이터 포인트를 나타냅니다.
    # 따라서, self.sma1[-1] < self.sma2[-1]는 바로 이전 시점에서 단기 SMA가 장기 SMA보다 낮았다는 것을 의미하며,
    # self.sma1[0] >= self.sma2[0]는 현재 시점에서 단기 SMA가 장기 SMA보다 같거나 크다는 것을 의미합니다.
    # 이 두 조건을 함께 사용하면, **단기 SMA가 장기 SMA를 방금 막 돌파한 상황(골든 크로스)**을 탐지할 수 있습니다.
    def next(self):
        if not self.position: #포지션이 없을 경우
            # 단기 SMA가 장기 SMA를 위로 돌파했을 경우
            if self.sma1[0] >= self.sma2[0] and self.sma1[-1] < self.sma2[-1]:
                order_size = math.floor(self.broker.get_value() / self.datas[0].close * 0.99)
                self.buy(size=order_size)

        # 장기 SMA가 단기 SMA를 아래로 돌파했을 경우
        elif self.sma1[0] <= self.sma2[0] and self.sma1[-1] > self.sma2[-1]:
            self.close()

    def log(self, message):
        print(message)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        cur_date = None
        if order.status in [order.Completed]:
            cur_date = order.data.datetime.date(0)
            if order.isbuy():
                self.log(
                    f'{cur_date} [매수 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')
            elif order.issell():
                self.log(
                    f'{cur_date} [매도 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log(
                f'{cur_date} 주문이 거부되었습니다. 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')



In [4]:
if __name__ == '__main__':

    cerebro = bt.Cerebro()
    cerebro.addstrategy(SmaCross)
    cerebro.broker.setcommission(commission=0.003) # 수수료 0.3%
    cerebro.broker.setcash(10_000_000)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    data = yf.download('BTC-USD', start='2020-01-01', end='2024-12-17')
    # 열 이름 전처리
    data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]
    data_bt = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(data_bt)
    cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print(f'수익률: {cerebro.broker.getvalue() / 10_000_000 * 100:.2f}%')


Starting Portfolio Value: 10000000.00


[*********************100%***********************]  1 of 1 completed


2021-09-02 [매수 주문 실행] 종목:  	 수량: 202 	 가격: 48807.85
2022-01-07 [매도 주문 실행] 종목:  	 수량: -202 	 가격: 43153.57
2023-01-29 [매수 주문 실행] 종목:  	 수량: 378 	 가격: 23031.45
2023-09-04 [매도 주문 실행] 종목:  	 수량: -378 	 가격: 25968.17
2023-10-26 [매수 주문 실행] 종목:  	 수량: 282 	 가격: 34504.29
2024-08-18 [매도 주문 실행] 종목:  	 수량: -282 	 가격: 59468.13
2024-10-18 [매수 주문 실행] 종목:  	 수량: 247 	 가격: 67419.11
Final Portfolio Value: 26303788.44
수익률: 263.04%


스탑로스 10% 설정

In [5]:
from datetime import datetime
import backtrader as bt
import math
import yfinance as yf

# SMA 전략
class SmaCrossStopLoss(bt.Strategy):
    params = dict(
        pfast=30,  # 단기 SMA
        pslow=200,  # 장기 SMA
        stop_loss=0.1 # 손절 비율(10%)
    )

    def __init__(self):
        self.sma1 = bt.ind.SMA(period=self.p.pfast) # 단기 SMA
        self.sma2 = bt.ind.SMA(period=self.p.pslow) # 장기 SMA
        self.order = None # 매수 신호 추적 변수
        self.buy_price = None # 매수 가격 저장

    def next(self):
        if not self.position: #포지션이 없을 경우
            # 단기 SMA가 장기 SMA를 위로 돌파했을 경우
            if self.sma1[0] >= self.sma2[0] and self.sma1[-1] < self.sma2[-1]:
                order_size = math.floor(self.broker.get_value() / self.datas[0].close * 0.99)
                self.buy(size=order_size)
                self.buy_price = self.data.close[0]
        else:
            # 장기 SMA가 단기 SMA를 아래로 돌파했거나 매수가 대비 10% 하락한 경우
            stop_loss_price = self.buy_price * (1 - self.params.stop_loss) # 손절가 계산
            if (self.sma1[0] <= self.sma2[0] and self.sma1[-1] > self.sma2[-1]) or self.data.close[0] < stop_loss_price:
                self.close()
                self.buy_price = None

    def log(self, message):
        print(message)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        cur_date = None
        if order.status in [order.Completed]:
            cur_date = order.data.datetime.date(0)
            if order.isbuy():
                self.log(
                    f'{cur_date} [매수 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')
            elif order.issell():
                self.log(
                    f'{cur_date} [매도 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log(
                f'{cur_date} 주문이 거부되었습니다. 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')



In [6]:
if __name__ == '__main__':

    cerebro = bt.Cerebro()
    cerebro.addstrategy(SmaCrossStopLoss)
    cerebro.broker.setcommission(commission=0.003) # 수수료 0.3%
    cerebro.broker.setcash(10_000_000)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    data = yf.download('BTC-USD', start='2020-01-01', end='2024-12-17')
    # 열 이름 전처리
    data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]
    data_bt = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(data_bt)
    cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print(f'수익률: {cerebro.broker.getvalue() / 10_000_000 * 100:.2f}%')


[*********************100%***********************]  1 of 1 completed

Starting Portfolio Value: 10000000.00


2021-09-02 [매수 주문 실행] 종목:  	 수량: 202 	 가격: 48807.85
2021-09-21 [매도 주문 실행] 종목:  	 수량: -202 	 가격: 43012.23
2023-01-29 [매수 주문 실행] 종목:  	 수량: 377 	 가격: 23031.45
2023-03-10 [매도 주문 실행] 종목:  	 수량: -377 	 가격: 20367.00
2023-10-26 [매수 주문 실행] 종목:  	 수량: 221 	 가격: 34504.29
2024-08-18 [매도 주문 실행] 종목:  	 수량: -221 	 가격: 59468.13
2024-10-18 [매수 주문 실행] 종목:  	 수량: 193 	 가격: 67419.11
Final Portfolio Value: 20587580.58
수익률: 205.88%


스탑로스 10%로 설정 + 윌리엄스의 리스크 고정

In [13]:
from datetime import datetime
import backtrader as bt
import math
import yfinance as yf

# SMA 전략
class SmaCrossStopLossRiskFixed(bt.Strategy):
    params = dict(
        pfast=30,  # 단기 SMA
        pslow=200,  # 장기 SMA
        stop_loss=0.1, # 손절 비율(10%)
        risk_percent=0.03,    # 리스크 비율 (3%)
    )

    def __init__(self):
        self.sma1 = bt.ind.SMA(period=self.p.pfast) # 단기 SMA
        self.sma2 = bt.ind.SMA(period=self.p.pslow) # 장기 SMA
        self.order = None # 매수 신호 추적 변수
        self.buy_price = None # 매수 가격 저장

    def next(self):
        if not self.position: #포지션이 없을 경우
            # 단기 SMA가 장기 SMA를 위로 돌파했을 경우
            if self.sma1[0] >= self.sma2[0] and self.sma1[-1] < self.sma2[-1]:
                self.buy_price = self.data.close[0]
                # 계좌 가치의 risk_percent 만큼의 리스크 금액 산정
                risk_amount = self.broker.getvalue() * self.params.risk_percent
                # 손절가: 매수 가격의 (1 - stop_loss) 비율
                stop_loss_price = self.buy_price * (1 - self.params.stop_loss)
                # 포지션 사이즈 계산 (리스크 금액 / (매수가 - 손절가))
                position_size = risk_amount / (self.buy_price - stop_loss_price)
                position_size = (position_size)
                if position_size <= 0:
                    return

                # 브라켓 주문 사용: 매수 주문과 동시에 스탑로스 주문을 부모 주문에 연계
                self.buy(size=position_size)

        else:
            # 장기 SMA가 단기 SMA를 아래로 돌파했거나 매수가 대비 10% 하락한 경우
            stop_loss_price = self.buy_price * (1 - self.params.stop_loss) # 손절가 계산
            if (self.sma1[0] <= self.sma2[0] and self.sma1[-1] > self.sma2[-1]) or self.data.close[0] < stop_loss_price:
                self.close()
                self.buy_price = None

    def log(self, message):
        print(message)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        cur_date = None
        if order.status in [order.Completed]:
            cur_date = order.data.datetime.date(0)
            if order.isbuy():
                self.log(
                    f'{cur_date} [매수 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')
            elif order.issell():
                self.log(
                    f'{cur_date} [매도 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log(
                f'{cur_date} 주문이 거부되었습니다. 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')



In [14]:
if __name__ == '__main__':

    cerebro = bt.Cerebro()
    cerebro.addstrategy(SmaCrossStopLossRiskFixed)
    cerebro.broker.setcommission(commission=0.003) # 수수료 0.3%
    cerebro.broker.setcash(10_000_000)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    data = yf.download('BTC-USD', start='2020-01-01', end='2024-12-17')
    # 열 이름 전처리
    data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]
    data_bt = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(data_bt)
    cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print(f'수익률: {cerebro.broker.getvalue() / 10_000_000 * 100:.2f}%')


[*********************100%***********************]  1 of 1 completed

Starting Portfolio Value: 10000000.00


2021-09-02 [매수 주문 실행] 종목:  	 수량: 61.41622455115174 	 가격: 48807.85
2021-09-21 [매도 주문 실행] 종목:  	 수량: -61.41622455115174 	 가격: 43012.23
2023-01-29 [매수 주문 실행] 종목:  	 수량: 125.40185006821096 	 가격: 23031.45
2023-03-10 [매도 주문 실행] 종목:  	 수량: -125.40185006821096 	 가격: 20367.00
2023-10-26 [매수 주문 실행] 종목:  	 수량: 80.66022556623481 	 가격: 34504.29
2024-08-18 [매도 주문 실행] 종목:  	 수량: -80.66022556623481 	 가격: 59468.13
2024-10-18 [매수 주문 실행] 종목:  	 수량: 50.152350511709216 	 가격: 67419.11
Final Portfolio Value: 13193803.12
수익률: 131.94%
